In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
to_categorical = tf.keras.utils.to_categorical
ReduceLROnPlateau = tf.keras.callbacks.ReduceLROnPlateau
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator

from sklearn.metrics import classification_report
import plotly
import plotly.graph_objs as go
import plotly.figure_factory as ff
import numpy as np
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix

In [6]:
img_rows, img_cols = 1024, 1024
batch_size = 16
epochs = 100
num_classes = 3
seed = 42

input_shape = (img_rows, img_cols, 3)

In [7]:
datagen = ImageDataGenerator(rescale = 1./255,
                             horizontal_flip = True,
                             zoom_range=0.3,
                             shear_range = 0.2,
                             rotation_range=0.3)

val_test_datagen = ImageDataGenerator(rescale = 1./255)

### Data

In [8]:
train_it = datagen.flow_from_directory('/content/drive/MyDrive/640710819_Projects/Tongue Classification/model 2/dataset/train',
                                       target_size=(img_rows, img_cols),
                                       batch_size=batch_size,
                                       class_mode='categorical',
                                       shuffle=True,
                                       seed = seed)

Found 178 images belonging to 3 classes.


In [9]:
val_it = val_test_datagen.flow_from_directory('/content/drive/MyDrive/640710819_Projects/Tongue Classification/model 2/dataset/valid',
                                              target_size=(img_rows, img_cols),
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=False,
                                              seed = seed)

test_it = val_test_datagen.flow_from_directory('/content/drive/MyDrive/640710819_Projects/Tongue Classification/model 2/dataset/test',
                                               target_size=(img_rows, img_cols),
                                               batch_size=batch_size,
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed = seed)

Found 58 images belonging to 3 classes.
Found 93 images belonging to 3 classes.


In [10]:
labels = list(train_it.class_indices.keys())
labels

['black hairy', 'cancer', 'diabete']

In [11]:
Counter(train_it.classes)

Counter({0: 45, 1: 43, 2: 90})

In [12]:
print(f'Total augmented black hairy image: {Counter(train_it.classes)[0]*100}')
print(f'Total augmented cancer image: {Counter(train_it.classes)[1]*100}')
print(f'Total augmented diabete image: {Counter(train_it.classes)[2]*100}')

Total augmented black hairy image: 4500
Total augmented cancer image: 4300
Total augmented diabete image: 9000


In [13]:
Counter(val_it.classes)

Counter({0: 16, 1: 11, 2: 31})

In [14]:
Counter(test_it.classes)

Counter({0: 31, 1: 31, 2: 31})

In [15]:
weights = compute_class_weight('balanced',
                               classes = np.unique(train_it.classes),
                               y = train_it.classes)
weights

array([1.31851852, 1.37984496, 0.65925926])

In [16]:
cw = dict(zip(np.unique(train_it.classes), weights))
cw

{0: 1.3185185185185184, 1: 1.37984496124031, 2: 0.6592592592592592}

In [17]:
#Feature Extraction
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

#Image Classification
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Train

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 1022, 1022, 32)      │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 511, 511, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 509, 509, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 254, 254, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 252, 252, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 126, 126, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 124, 124, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 246016)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      31,490,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,564,195 (120.41 MB)

 Trainable params: 31,564,195 (120.41 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
STEP_SIZE_TRAIN=train_it.n//train_it.batch_size
STEP_SIZE_VALID=val_it.n//val_it.batch_size
STEP_SIZE_TEST=test_it.n//test_it.batch_size
import math

STEP_SIZE_TRAIN = math.ceil(train_it.n / train_it.batch_size)
STEP_SIZE_VALID = math.ceil(val_it.n / val_it.batch_size)

In [20]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience = 2, verbose=1,factor=0.1, min_lr=0.000001)
callbacks_list = [learning_rate_reduction]

In [ ]:
his = model.fit(
        train_it,
        validation_data=val_it,
        epochs=epochs,
        class_weight=cw,
        callbacks=callbacks_list,
        shuffle=True,
        steps_per_epoch=STEP_SIZE_TRAIN,
        validation_steps=STEP_SIZE_VALID)

In [21]:
his = model.fit(
    train_it,
    validation_data=val_it,
    epochs=epochs,
    class_weight=cw,
    callbacks=callbacks_list,
    steps_per_epoch=None,
    validation_steps=None
)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


12/12 ━━━━━━━━━━━━━━━━━━━━ 175s 8s/step - accuracy: 0.5484 - loss: 1.2771 - val_accuracy: 0.7241 - val_loss: 0.7111 - learning_rate: 0.0010
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.7248 - loss: 0.7652 - val_accuracy: 0.7759 - val_loss: 0.6603 - learning_rate: 0.0010
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.6879 - loss: 0.7803 - val_accuracy: 0.8103 - val_loss: 0.4594 - learning_rate: 0.0010
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.7819 - loss: 0.6182 - val_accuracy: 0.8621 - val_loss: 0.4689 - learning_rate: 0.0010
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.7826 - loss: 0.5704 - val_accuracy: 0.8621 - val_loss: 0.3730 - learning_rate: 0.0010
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.8233 - loss: 0.4799 - val_accuracy: 0.8103 - val_loss: 0.4733 - learning_rate: 0.0010
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8023 - loss: 0.4467
Epoch 7: ReduceLROnPl


### Evaluate

In [22]:
lossacc = model.evaluate(test_it)
print(f'Model 2 Loss : {lossacc[0]}')
print(f'Model 2 Accuracy : {lossacc[1]}')

6/6 ━━━━━━━━━━━━━━━━━━━━ 44s 9s/step - accuracy: 0.7697 - loss: 0.5948
Model 2 Loss : 0.42496928572654724
Model 2 Accuracy : 0.8387096524238586


In [23]:
h1 = go.Scatter(y=his.history['loss'],
                    mode="lines",
                    line=dict(
                        width=2,
                        color='blue'),
                        name="loss"
                   )
h2 = go.Scatter(y=his.history['val_loss'],
                    mode="lines",
                    line=dict(
                        width=2,
                        color='red'),
                        name="val_loss"
                   )

data = [h1,h2]
layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data, layout=layout1)
plotly.offline.iplot(fig1)

In [24]:
h1 = go.Scatter(y=his.history['accuracy'],
                    mode="lines", line=dict(
                    width=2,
                    color='blue'),
                    name="acc"
                   )
h2 = go.Scatter(y=his.history['val_accuracy'],
                    mode="lines", line=dict(
                    width=2,
                    color='red'),
                    name="val_acc"
                   )

data = [h1,h2]
layout1 = go.Layout(title='Accuracy',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
plotly.offline.iplot(fig1)

In [25]:
predicted_score = model.predict(test_it, verbose=1)
predicted_classes = np.argmax(predicted_score, axis=1)

6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 485ms/step


In [26]:
def cm_plot(cm, labels):
    x = labels
    y = labels

    z_text = [[str(y) for y in x] for x in cm]
    fig = ff.create_annotated_heatmap(cm, x=x, y=y, annotation_text=z_text, colorscale='blues')

    fig.update_layout(title_text='Confusion Matrix')

    fig.add_annotation(dict(font=dict(color="black",size=13),
                            x=0.5,
                            y=-0.15,
                            showarrow=False,
                            text="Predicted Value",
                            xref="paper",
                            yref="paper"
                           ))

    fig.add_annotation(dict(font=dict(color="black",size=13),
                            x=-0.20,
                            y=0.5,
                            showarrow=False,
                            text="Real Value",
                            textangle=-90,
                            xref="paper",
                            yref="paper"
                           ))

    fig.update_layout(margin=dict(t=50, l=200))
    fig['layout']['yaxis']['autorange'] = "reversed"

    fig['data'][0]['showscale'] = True
    fig.show()

In [27]:
cm = confusion_matrix(test_it.classes, predicted_classes)
print(cm)
cm_plot(cm, labels)

[[21  5  5]
 [ 3 27  1]
 [ 0  1 30]]


In [28]:
report = classification_report(test_it.classes, predicted_classes, target_names=labels, digits=4)

print(report)

              precision    recall  f1-score   support

 black hairy     0.8750    0.6774    0.7636        31
      cancer     0.8182    0.8710    0.8438        31
     diabete     0.8333    0.9677    0.8955        31

    accuracy                         0.8387        93
   macro avg     0.8422    0.8387    0.8343        93
weighted avg     0.8422    0.8387    0.8343        93



### Save

In [29]:
model.save('/content/drive/MyDrive/640710819_Projects/Tongue Classification/model 2/model_2_16b100e_9_11_2024.keras')
model.save('/content/drive/MyDrive/640710819_Projects/Tongue Classification/model 2/model_2_16b100e_11_11_2024.h5')

In [30]:
import pickle as p
his_model_name = '/content/drive/MyDrive/640710819_Projects/Tongue Classification/model 2/model_2_16b100e_9_11_2024_history'
with open(his_model_name, 'wb') as file:
  p.dump(his.history, file)